In [13]:
from pathlib import Path
import pandas as pd
import json
import geopandas as gpd
import fiona

In [2]:
instance_path = Path(r"D:\AIC backup data\Experiment Data\DARP\final\Instances\NYC\instances\start_18-00\duration_16_h\max_delay_05_min")
out_path = Path(r"C:\Google Drive\AIC Experiment Data\NYC SiMoD")

# Vehicle conversion

In [9]:
vehicle_path = instance_path / "vehicles.csv"
vehicles = pd.read_csv(vehicle_path, sep="\t", names=["node_index", "capacity"])
vehicles

,node_index,capacity
0,91589,4
1,11598,4
2,50951,4
3,36509,4
4,85225,4
...,...,...
6456,11427,4
6457,31525,4
6458,59746,4
6459,87326,4


In [10]:
veh_out = []
for index, node_index, capacity in zip(vehicles.index, vehicles.node_index, vehicles.capacity):
    veh_out.append({
        "id": index,
        "node_index": node_index,
        "capacity": capacity
    })
veh_out

[{'id': 0, 'node_index': 91589, 'capacity': 4},
 {'id': 1, 'node_index': 11598, 'capacity': 4},
 {'id': 2, 'node_index': 50951, 'capacity': 4},
 {'id': 3, 'node_index': 36509, 'capacity': 4},
 {'id': 4, 'node_index': 85225, 'capacity': 4},
 {'id': 5, 'node_index': 107044, 'capacity': 4},
 {'id': 6, 'node_index': 82978, 'capacity': 4},
 {'id': 7, 'node_index': 31191, 'capacity': 4},
 {'id': 8, 'node_index': 7141, 'capacity': 4},
 {'id': 9, 'node_index': 8532, 'capacity': 4},
 {'id': 10, 'node_index': 61002, 'capacity': 4},
 {'id': 11, 'node_index': 39096, 'capacity': 4},
 {'id': 12, 'node_index': 43844, 'capacity': 4},
 {'id': 13, 'node_index': 74035, 'capacity': 4},
 {'id': 14, 'node_index': 108887, 'capacity': 4},
 {'id': 15, 'node_index': 40769, 'capacity': 4},
 {'id': 16, 'node_index': 83003, 'capacity': 4},
 {'id': 17, 'node_index': 35898, 'capacity': 4},
 {'id': 18, 'node_index': 10929, 'capacity': 4},
 {'id': 19, 'node_index': 35964, 'capacity': 4},
 {'id': 20, 'node_index': 1646

In [13]:
veh_out_path = out_path / "vehicles.json"
json.dump(veh_out, open(veh_out_path, "w"), indent=4)

# Request conversion

In [15]:
request_path = instance_path / "trips.csv"
requests = pd.read_csv(request_path, sep="\t")
requests

,time_ms,origin,dest,min_travel_time
0,64800000,56751,31517,228
1,64800000,38748,33231,455
2,64800000,20074,60568,208
3,64800000,10157,40154,660
4,64800000,15578,40790,755
...,...,...,...,...
198722,86398000,8020,14781,1690
198723,86398000,16196,38738,884
198724,86398000,59246,90983,1327
198725,86399000,75928,77850,293


In [21]:
requests_out_path = out_path / "trips.csv"
out = requests.rename(columns={
    "time_ms": "Announcement_Time",
    "origin": "From",
    "dest": "To"
})
out.drop(columns=["min_travel_time"], inplace=True)
out.to_csv(requests_out_path, index_label="id")

# Map

In [3]:
map_path = Path(r"D:\AIC backup data\Experiment Data\DARP\final\Instances\NYC/map/")
out_map_path = out_path / "maps"

## Nodes

In [10]:
nodes_path = map_path / "nodes.csv"
nodes = pd.read_csv(nodes_path, sep="\t")
nodes

,id,db_id,x,y
0,72502,39076461,-73.794627,40.786409
1,72503,39076490,-73.757091,40.762429
2,72504,39076504,-73.744164,40.753467
3,7519,42421728,-73.960043,40.798049
4,7520,42421731,-73.961474,40.798645
...,...,...,...,...
113406,113408,6578689015,-73.959634,40.736733
113407,113409,6578689034,-73.958943,40.736046
113408,72505,6579197085,-74.038418,40.711402
113409,72506,6579245770,-74.029593,40.731043


In [12]:
out_node_path = out_map_path / "nodes.geojson"
nodes_out = {
    "type": "FeatureCollection",
    "features": []
}
for index, id, db_id, x, y in zip(nodes.index, nodes.id, nodes.db_id, nodes.x, nodes.y):
    nodes_out["features"].append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [x, y]
        },
        "properties": {
            "node_id": id,
            "index": index,
            "db_id": db_id
        }
    })
json.dump(nodes_out, open(out_node_path, "w"), indent=4)

## Edges

In [4]:
edges_path = map_path / "edges.csv"
edges = pd.read_csv(edges_path, sep="\t")
edges[:10]

,u,v,db_id_from,db_id_to,length,speed
0,55830,72502,274281354,39076461,808.969535,78.129359
1,56497,72503,277672389,39076490,342.659220,89.172156
2,57848,72504,462122262,39076504,219.100576,90.419384
3,84355,7519,3602679383,42421728,66.021733,31.757588
4,84352,7519,3602678205,42421728,63.662611,11.417022
5,38620,7519,42432736,42421728,85.918359,31.127933
6,84352,7520,3602678205,42421731,74.020642,26.800245
7,7812,7520,42437916,42421731,84.513003,18.494581
8,38621,7520,42432737,42421731,86.381875,26.840639
9,7521,7520,42421737,42421731,136.991706,23.349896


In [5]:
edges_path_shp = map_path / "shapefiles/edges.shp"
edges_shp = gpd.read_file(edges_path_shp)
edges_shp[:10]

,u,v,db_id_from,db_id_to,length,speed,geometry
0,55830,72502,274281354,39076461,808.969535,78.129359,"MULTILINESTRING ((-73.79895 40.78586, -73.7976..."
1,56497,72503,277672389,39076490,342.659220,89.172156,"MULTILINESTRING ((-73.75389 40.76058, -73.7544..."
2,57848,72504,462122262,39076504,219.100576,90.419384,"MULTILINESTRING ((-73.74276 40.75181, -73.7435..."
3,84355,7519,3602679383,42421728,66.021733,31.757588,"LINESTRING (-73.96042 40.79753, -73.96004 40.7..."
4,84352,7519,3602678205,42421728,63.662611,11.417022,"MULTILINESTRING ((-73.96042 40.79821, -73.9600..."
5,38620,7519,42432736,42421728,85.918359,31.127933,"LINESTRING (-73.95955 40.79872, -73.96004 40.7..."
6,84352,7520,3602678205,42421731,74.020642,26.800245,"MULTILINESTRING ((-73.96071 40.79832, -73.9610..."
7,7812,7520,42437916,42421731,84.513003,18.494581,"LINESTRING (-73.96196 40.79798, -73.96147 40.7..."
8,38621,7520,42432737,42421731,86.381875,26.840639,"LINESTRING (-73.96098 40.79932, -73.96147 40.7..."
9,7521,7520,42421737,42421731,136.991706,23.349896,"MULTILINESTRING ((-73.96187 40.79881, -73.9614..."


In [11]:
str(edges_shp.iloc[0].geometry)

'MULTILINESTRING ((-73.7989488 40.7858568, -73.7976382 40.7858354), (-73.8010644 40.785937499999996, -73.7989488 40.7858568), (-73.8018979 40.7859823, -73.8010644 40.785937499999996), (-73.7954752 40.7861664, -73.7946273 40.7864093), (-73.8040942 40.786126, -73.803413 40.786077299999995), (-73.7976382 40.7858354, -73.7965824 40.78593899999999), (-73.8025763 40.78602299999999, -73.8023065 40.786006), (-73.8023065 40.786006, -73.8018979 40.7859823), (-73.7965824 40.78593899999999, -73.7954752 40.7861664), (-73.803413 40.786077299999995, -73.8025763 40.78602299999999))'

In [9]:
edged_shp_exploded = edges_shp.explode()
edged_shp_exploded

C:\Users\david\AppData\Local\Temp\ipykernel_7372\338411800.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  edged_shp_exploded = edges_shp.explode()


u       v  db_id_from    db_id_to       length      speed  \
0      0   55830   72502   274281354    39076461   808.969535  78.129359   
       1   55830   72502   274281354    39076461   808.969535  78.129359   
       2   55830   72502   274281354    39076461   808.969535  78.129359   
       3   55830   72502   274281354    39076461   808.969535  78.129359   
       4   55830   72502   274281354    39076461   808.969535  78.129359   
...          ...     ...         ...         ...          ...        ...   
281277 30  36402  113410    42423514  6579287741  2284.464914  29.614844   
       31  36402  113410    42423514  6579287741  2284.464914  29.614844   
       32  36402  113410    42423514  6579287741  2284.464914  29.614844   
       33  36402  113410    42423514  6579287741  2284.464914  29.614844   
       34  36402  113410    42423514  6579287741  2284.464914  29.614844   

                                                    geometry  
0      0   LINESTRING (-73.79895 40.78586, -73.79764 40.7...  
       1   LINESTRING (-73.80106 40.78594, -73.79895 40.7...  
       2   LINESTRING (-73.80190 40.78598, -73.80106 40.7...  
       3   LINESTRING (-73.79548 40.78617, -73.79463 40.7...  
       4   LINESTRING (-73.80409 40.78613, -73.80341 40.7...  
...                                                      ...  
281277 30  LINESTRING (-74.01547 40.72669, -74.01564 40.7...  
       31  LINESTRING (-74.02109 40.72760, -74.02616 40.7...  
       32  LINESTRING (-74.00791 40.72591, -74.00804 40.7...  
       33  LINESTRING (-74.00779 40.72589, -74.00791 40.7...  
       34  LINESTRING (-74.00804 40.72593, -74.01517 40.7...  

[640398 rows x 7 columns]

In [15]:
edges = fiona.open(edges_path_shp)
edges[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('u', 55830),
              ('v', 72502),
              ('db_id_from', 274281354),
              ('db_id_to', 39076461),
              ('length', 808.9695354136534),
              ('speed', 78.12935892740886)]),
 'geometry': {'type': 'MultiLineString',
  'coordinates': [[(-73.7989488, 40.7858568), (-73.7976382, 40.7858354)],
   [(-73.8010644, 40.785937499999996), (-73.7989488, 40.7858568)],
   [(-73.8018979, 40.7859823), (-73.8010644, 40.785937499999996)],
   [(-73.7954752, 40.7861664), (-73.7946273, 40.7864093)],
   [(-73.8040942, 40.786126), (-73.803413, 40.786077299999995)],
   [(-73.7976382, 40.7858354), (-73.7965824, 40.78593899999999)],
   [(-73.8025763, 40.78602299999999), (-73.8023065, 40.786006)],
   [(-73.8023065, 40.786006), (-73.8018979, 40.7859823)],
   [(-73.7965824, 40.78593899999999), (-73.7954752, 40.7861664)],
   [(-73.803413, 40.786077299999995), (-73.8025763, 40.78602299999999)]]}}

In [8]:
out_edge_path = out_map_path / "edges.geojson"
edges_out = {
    "type": "FeatureCollection",
    "features": []
}
for index, u, v, length, speed, geometry in zip(edges_shp.index, edges_shp.u, edges_shp.v, edges_shp.length, edges_shp.speed, edges_shp.geometry):
    coords = geometry.coords if geometry.geom_type == "LineString" else [list(p.coords)[0] for p in geometry.geoms]
    coords = [[c[0], c[1]] for c in coords]
    edges_out["features"].append({
        "type": "Feature",
        "id": index,
        "geometry": {
            "type": "LineString",
            "coordinates": coords
        },
        "properties": {
            "length": length,
            "maxspeed": speed,
            "from_id": u,
            "to_id": v,
            "lanes": 1
        }
    })
json.dump(edges_out, open(out_edge_path, "w"), indent=4)
                

C:\Users\david\AppData\Local\Temp\ipykernel_32808\2985367877.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  for index, u, v, length, speed, geometry in zip(edges_shp.index, edges_shp.u, edges_shp.v, edges_shp.length, edges_shp.speed, edges_shp.geometry):
